In [ ]:
pip install kmapper

In [52]:
import pandas as pd
import kmapper as km
from kmapper.jupyter import display
import sklearn
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [37]:
soiAnom = pd.read_csv("/content/gdrive/MyDrive/soiAnomaly.csv")
soiStd = pd.read_csv("/content/gdrive/MyDrive/soiStandardized.csv")

ONI =  pd.read_csv("/content/gdrive/MyDrive/ONI.csv")

sst = pd.read_csv("/content/gdrive/MyDrive/sst.csv")

olrOg = pd.read_csv("/content/gdrive/MyDrive/olrOriginal.csv")
olrStd = pd.read_csv("/content/gdrive/MyDrive/olrStandardized.csv")
olrAnom = pd.read_csv("/content/gdrive/MyDrive/olrAnomaly.csv")

In [38]:
month_to_number = {
    "JAN":1, 
                 "FEB":2, 
                 "MAR":3,
                 "APR":4,
                 "MAY":5,
                 "JUN":6,
                 "JUL":7,
                 "AUG":8,
                 "SEP":9,
                 "OCT":10,
                 "NOV":11,
                 "DEC":12
                }
SEAS_to_mon = {
    "NDJ":1, 
                 "DJF":2, 
                 "JFM":3,
                 "FMA":4,
                 "MAM":5,
                 "AMJ":6,
                 "MJJ":7,
                 "JJA":8,
                 "JAS":9,
                 "ASO":10,
                 "SON":11,
                 "OND":12
                }

In [39]:
def dfPreparation(df):
  df= df.melt(id_vars="YEAR",var_name="MON",value_name='value',)
  df['MON'] = pd.Categorical(df['MON'],categories=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'],ordered=True)
  df=  df.sort_values(["YEAR","MON"])
  df["MON"] = df['MON'].map(month_to_number)
  return df

In [40]:
soiAnom = dfPreparation(soiAnom)
soiStd = dfPreparation(soiStd)

SOI = pd.merge(soiAnom, soiStd, on = ["YEAR", "MON"])
SOI.rename(columns = {'value_x':'SOI_Anomalias','value_y':'SOI_Estandarizada'}, inplace = True)

olrOg = dfPreparation(olrOg)
olrStd = dfPreparation(olrStd)
olrAnom = dfPreparation(olrAnom)

olr1 = pd.merge(olrOg, olrStd, on = ["YEAR", "MON"])
OLR = pd.merge(olr1, olrAnom, on = ["YEAR", "MON"])
OLR.rename(columns = {'value_x':'OLR_Original','value_y':'OLR_Estandarizada','value':'OLR_Anomalias'}, inplace = True)

In [46]:
OLR.corr()

<ipython-input-46-a65440c5a625>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  OLR.corr()


,YEAR,OLR_Original,OLR_Estandarizada,OLR_Anomalias
YEAR,1.000000,0.046221,0.037566,0.047407
OLR_Original,0.046221,1.000000,0.997698,0.999578
OLR_Estandarizada,0.037566,0.997698,1.000000,0.997892
OLR_Anomalias,0.047407,0.999578,0.997892,1.000000


In [41]:
ONI.loc[ONI.SEAS == 'NDJ', 'YR'] = ONI['YR'] + 1
ONI["MON"] = ONI['SEAS'].map(SEAS_to_mon)
ONI.rename(columns = {'YR':'YEAR', 'TOTAL':'TOTAL_ONI','ANOM':'ANOM_ONI'}, inplace = True)
ONI = ONI.drop(columns=['SEAS'])

In [42]:
sst.rename(columns = {'YR':'YEAR'}, inplace = True)

In [43]:
ENSO =  pd.merge(sst, SOI, on = ["YEAR", "MON"])
ENSO =  pd.merge(ENSO, OLR, on = ["YEAR", "MON"])
ENSO =  pd.merge(ENSO, ONI, on = ["YEAR", "MON"])
ENSO

,YEAR,MON,NINO1+2,ANOM.12,NINO3,ANOM.3,NINO4,ANOM.4,NINO3.4,ANOM3.4,SOI_Anomalias,SOI_Estandarizada,OLR_Original,OLR_Estandarizada,OLR_Anomalias,TOTAL_ONI,ANOM_ONI
0,1982,1,24.28,-0.24,25.84,0.17,28.01,-0.21,26.65,0.08,2.0,1.2,49.5,0.9,14.7,26.46,-0.08
1,1982,2,25.38,-0.72,26.26,-0.11,27.99,-0.11,26.54,-0.20,0.4,0.3,36.9,0.1,2.0,26.56,-0.05
2,1982,3,25.22,-1.38,26.92,-0.25,28.18,-0.05,27.09,-0.14,1.0,0.6,28.4,-0.6,-8.6,26.90,0.07
3,1982,4,24.57,-1.16,27.52,-0.05,28.61,0.10,27.83,0.02,0.1,0.1,37.1,0.0,-0.2,27.35,0.19
4,1982,5,24.00,-0.62,27.70,0.49,29.19,0.40,28.37,0.49,-0.6,-0.3,49.7,0.4,7.0,27.95,0.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,2022,12,22.54,-0.30,24.48,-0.78,27.71,-0.73,25.81,-0.84,3.5,2.1,63.3,1.7,26.2,25.76,-0.92
492,2023,1,24.27,-0.24,25.17,-0.50,27.62,-0.60,25.88,-0.69,2.3,1.4,63.3,1.8,28.5,25.80,-0.82
493,2023,2,26.80,0.71,26.25,-0.13,27.58,-0.52,26.29,-0.44,2.3,1.4,56.9,1.4,22.0,25.97,-0.67
494,2023,3,28.12,1.52,27.52,0.36,28.09,-0.14,27.23,-0.01,0.3,0.2,58.7,1.4,21.7,26.44,-0.43


In [48]:
ENSO1 = ENSO.drop(columns=['NINO1+2', 'ANOM.12','SOI_Estandarizada','OLR_Original','OLR_Estandarizada'])
ENSO1

,YEAR,MON,NINO3,ANOM.3,NINO4,ANOM.4,NINO3.4,ANOM3.4,SOI_Anomalias,OLR_Anomalias,TOTAL_ONI,ANOM_ONI
0,1982,1,25.84,0.17,28.01,-0.21,26.65,0.08,2.0,14.7,26.46,-0.08
1,1982,2,26.26,-0.11,27.99,-0.11,26.54,-0.20,0.4,2.0,26.56,-0.05
2,1982,3,26.92,-0.25,28.18,-0.05,27.09,-0.14,1.0,-8.6,26.90,0.07
3,1982,4,27.52,-0.05,28.61,0.10,27.83,0.02,0.1,-0.2,27.35,0.19
4,1982,5,27.70,0.49,29.19,0.40,28.37,0.49,-0.6,7.0,27.95,0.47
...,...,...,...,...,...,...,...,...,...,...,...,...
491,2022,12,24.48,-0.78,27.71,-0.73,25.81,-0.84,3.5,26.2,25.76,-0.92
492,2023,1,25.17,-0.50,27.62,-0.60,25.88,-0.69,2.3,28.5,25.80,-0.82
493,2023,2,26.25,-0.13,27.58,-0.52,26.29,-0.44,2.3,22.0,25.97,-0.67
494,2023,3,27.52,0.36,28.09,-0.14,27.23,-0.01,0.3,21.7,26.44,-0.43


In [49]:
fecha = ENSO1['YEAR'].astype(str) + '-' + ENSO1['MON'].astype(str)

In [50]:
color = []
for i in ENSO1["ANOM_ONI"]:
  if i > 0.5:
    color.append(1)
  elif i < -0.5:
    color.append(-1)
  else:
    color.append(0)

In [53]:
data = ENSO1.to_numpy()
data = np.delete(data, 0, axis=1)
data = np.delete(data, 0, axis=1)
data

array([[25.84, 0.17, 28.01, ..., 14.7, 26.46, -0.08],
       [26.26, -0.11, 27.99, ..., 2.0, 26.56, -0.05],
       [26.92, -0.25, 28.18, ..., -8.6, 26.9, 0.07],
       ...,
       [26.25, -0.13, 27.58, ..., 22.0, 25.97, -0.67],
       [27.52, 0.36, 28.09, ..., 21.7, 26.44, -0.43],
       [28.02, 0.44, 28.81, ..., 9.9, 27.14, -0.16]], dtype=object)

In [54]:
mapper = km.KeplerMapper(verbose=1)

KeplerMapper(verbose=1)


In [56]:
projected_data = mapper.fit_transform(data, projection=[9])

..Composing projection pipeline of length 1:
	Projections: [9]
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (496, 10)

..Projecting data using: [9]

..Scaling with: MinMaxScaler()



In [57]:
cover = km.Cover(n_cubes = 4, perc_overlap = 0.1)

In [58]:
graph = mapper.map(projected_data, data, cover=cover,clusterer=KMeans(n_clusters=3))

Mapping on data shaped (496, 10) using lens shaped (496, 1)

Creating 4 hypercubes.

Created 10 edges and 12 nodes in 0:00:00.117351.


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [59]:
graph['meta_data']

{'projection': '[9]',
 'n_cubes': 4,
 'perc_overlap': 0.1,
 'clusterer': 'KMeans(n_clusters=3)',
 'scaler': 'MinMaxScaler()'}

In [60]:
fileID = 'projection=' + graph['meta_data']['projection'].split('(')[0] + '_' + \
'n_cubes=' + str(graph['meta_data']['n_cubes']) + '_' + \
'perc_overlap=' + str(graph['meta_data']['perc_overlap']) + '_' + \
'clusterer=' + graph['meta_data']['clusterer'].split('(')[0] + '_' + \
'scaler=' + graph['meta_data']['scaler'].split('(')[0]

In [61]:
mapper.visualize(graph, 
                title=fileID,
                custom_tooltips = fecha,
                color_values = color,
                color_function_name = 'Fenomeno, Niño, Niña o Ninguno',
                node_color_function=np.array(['average','std','sum','max','min']))

km.jupyter.display("mapper" + fileID + ".html")

Wrote visualization to: mapper_visualization_output.html


/usr/local/lib/python3.10/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")
